In [43]:
pip install google-auth google-analytics-data

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [53]:
pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 249 kB 2.6 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [55]:
from google.auth import exceptions
from google.analytics.data_v1beta import BetaAnalyticsDataClient
from google.analytics.data_v1beta import types
from google.auth.transport.requests import Request
from google.oauth2 import service_account
import pandas as pd

In [47]:
import json

credentials_path = '/Users/vetybhakti/report-GA4/report-GA4/.env'

try:
    with open(credentials_path, 'r') as file:
        credentials_data = json.load(file)
        # Construct credentials
        credentials = service_account.Credentials.from_service_account_info(
            credentials_data,
            scopes=['https://www.googleapis.com/auth/analytics']  # Adjust the scope based on your needs
        )
except FileNotFoundError:
    print(f"Error: File not found at {credentials_path}")

In [48]:
# Authenticate with Google API
try:
    credentials.refresh(Request())
    client = BetaAnalyticsDataClient(credentials=credentials)
except Exception as e:
    print(f"Error: Unable to authenticate with Google API: {e}")
    # Add appropriate error handling or exit the script

In [49]:
# Year Month
# Construct the API request with 'yearMonth' dimension
request = types.RunReportRequest(
    property=f"properties/{property_id}",
    dimensions=[types.Dimension(name="yearMonth")],  # Change the dimension to 'yearMonth'
    metrics=[types.Metric(name="activeUsers")],
    date_ranges=[types.DateRange(start_date="2023-01-01", end_date="2024-02-29")]
)

# Execute the request
response = client.run_report(request)

# Process the response
for row in response.rows:
    year_month = row.dimension_values[0].value
    active_users = row.metric_values[0].value
    print(f"Year-Month: {year_month}, Active Users: {active_users}")
    
# Execute the request
response = client.run_report(request)

# Process the response and create DataFrame
data = []
for row in response.rows:
    year_month = row.dimension_values[0].value
    active_users = row.metric_values[0].value
    data.append({'Year-Month': year_month, 'Active Users': active_users})

# Create DataFrame and sort by 'Year-Month'
df_mua = pd.DataFrame(data).sort_values(by='Year-Month').reset_index(drop=True)
df_mua

Year-Month: 202308, Active Users: 421310
Year-Month: 202402, Active Users: 293144
Year-Month: 202307, Active Users: 281551
Year-Month: 202310, Active Users: 261163
Year-Month: 202306, Active Users: 245667
Year-Month: 202312, Active Users: 188148
Year-Month: 202311, Active Users: 176979
Year-Month: 202401, Active Users: 169119
Year-Month: 202309, Active Users: 140163
Year-Month: 202304, Active Users: 60372
Year-Month: 202305, Active Users: 55733
Year-Month: 202303, Active Users: 1


,Year-Month,Active Users
0,202303,1
1,202304,60372
2,202305,55733
3,202306,245667
4,202307,281551
5,202308,421310
6,202309,140163
7,202310,261163
8,202311,176979
9,202312,188148


In [54]:
df_mua.to_excel('mua.xlsx')

In [41]:
# Construct the API request
request = types.RunReportRequest(
    property=f"properties/{property_id}",
    dimensions=[
        types.Dimension(name="audienceId"),
        types.Dimension(name="yearMonth")
    ],
    metrics=[types.Metric(name="sessions")],
    date_ranges=[types.DateRange(start_date="2024-01-01", end_date="2024-01-31")]
)

# Execute the request
response = client.run_report(request)

# Process the response to see individual user activity
data = []
for row in response.rows:
    audience_id = row.dimension_values[0].value
    year_month = row.dimension_values[1].value
    sessions = row.metric_values[0].value
    data.append({'Audience ID': audience_id, 'Year-Month': year_month, 'Sessions': sessions})

# Create DataFrame and sort by 'Year-Month'
df = pd.DataFrame(data).sort_values(by='Year-Month').reset_index(drop=True)
df

,Audience ID,Year-Month,Sessions
0,4689351413,202401,209275
1,6287367354,202401,208627
2,6315622208,202401,104573
3,6289957303,202401,104506
4,6298358827,202401,86635
5,6287340496,202401,29921
6,6287373302,202401,6575
7,6287363908,202401,5625
8,4689351414,202401,2621
9,6670607610,202401,254
